# Modeling

## Load Packages

In [1]:
from operator import mod
from os import getcwd
from os.path import exists, join

import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

In [2]:
# pip install ydata_profiling

In [3]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import  GradientBoostingClassifier
# import xgboost as xgb
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC 
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import recall_score

from sklearn import tree
from sklearn.decomposition import PCA, SparsePCA

from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import json
import pickle
from IPython.display import Image
import warnings

In [4]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

In [5]:
import altair as alt
import random
import warnings

In [6]:
warnings.filterwarnings('ignore')

## Load Data

In [7]:
cdc_survey = pd.read_csv('../data/cdc_nhanes_survey_responses_clean.csv')

# filter to pregnant moms
cdc_survey_pmom = cdc_survey[cdc_survey['has_been_pregnant'] == 1]
cdc_survey_pmom

,SEQN,SMQ681,SMQ690A,SMQ710,SMQ720,SMQ725,SMQ690B,SMQ740,SMQ690C,SMQ770,...,monthly_poverty_index,age_with_angina_pectoris,age_liver_condition,age_range_first_menstrual_period,annual_healthcare_visit_count,have_liver_condition,type_of_work_done_last_week,weight_change_intentional,days_nicotine_substitute_used,pain_relief_from_cardio_recoverytime
8,109284,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
11,109290,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN
12,109291,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8.0,NaN,1.0,NaN,NaN,NaN
15,109295,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
18,109300,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10394,124802,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN
10395,124803,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,8.0,NaN,4.0,NaN,NaN,NaN
10401,124812,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,1.0
10402,124813,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN


## Training Step

### Features to Use

In [8]:
all_columns = [
    # Depression screener
    'little_interest_in_doing_things',
    'feeling_down_depressed_hopeless',
    'trouble_falling_or_staying_asleep',
    'feeling_tired_or_having_little_energy',
    'poor_appetitie_or_overeating',
    'feeling_bad_about_yourself',
    'trouble_concentrating',
    'moving_or_speaking_to_slowly_or_fast',
    'thoughts_you_would_be_better_off_dead',
    'difficult_doing_daytoday_tasks',
    # Alcohol & smoking
    'has_smoked_tabacco_last_5days',
    'alcoholic_drinks_past_12mo',    
    # Diet & Nutrition
    'how_healthy_is_your_diet',    
    'count_lost_10plus_pounds',
    'has_tried_to_lose_weight_12mo',       
    # Physical health & Medical History
    'count_days_seen_doctor_12mo',
    'duration_last_healthcare_visit',        
    'count_days_moderate_recreational_activity',   
    'count_minutes_moderate_recreational_activity',
    'count_minutes_moderate_sedentary_activity',
    'general_health_condition',    
    'has_diabetes',
    'has_overweight_diagnosis',         
    # Demographic data
    'food_security_level_household',   
    'food_security_level_adult',    
    'monthly_poverty_index_category',
    'monthly_poverty_index',
    'count_hours_worked_last_week',
    'age_in_years',   
    'education_level',
    'is_usa_born',    
    'has_health_insurance',
    'has_health_insurance_gap'   
]
len(all_columns)

33

In [9]:
# find columns where everything is null
# need to delete those
#cdc_survey_pmom = cdc_survey_pmom[all_columns]
#columns = cdc_survey_pmom.columns[cdc_survey_pmom.notnull().any() == False]
#columns

### Create Test & Train Datasets

In [10]:
def get_model_data(original_df, columns, test_size_prop=0.2):
    """
    Function to build feature & indicator matrices for both train & test.
    """
    
    # add target column (MDD)
    cols_to_use = columns.copy()
    cols_to_use.insert(0, 'MDD')
    
    df_to_use = original_df[cols_to_use]
    
    # Create test & train data
    x = df_to_use.iloc[:,1:].values
    y = df_to_use['MDD'].values
    
    # SimpleImputer() = fill in missing values
    # note imputer may drop columns if no values exist for it
    imputer = SimpleImputer(strategy='median')  
    x = imputer.fit_transform(x)

    # RobustScaler() = scale features to remove outliers
    trans = RobustScaler()
    x = trans.fit_transform(x)

    x_train, x_test, y_train, y_test = train_test_split(
        x, 
        y, 
        test_size=test_size_prop, 
        random_state=42
    ) 
    
    return x_train, x_test, y_train, y_test

## Test Step

In [11]:
def plot_confusion_matrix(y_test, pred_labels):
    """
    Function that displays a confusion matrix for provided true and predicted classes
    """
    #print(f'cover type 1 and type 2 total correct {np.sum(np.diag(metrics.confusion_matrix(y_test, pred_labels))[:2])}')

    cm = confusion_matrix(y_test, pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    fig, ax = plt.subplots(figsize=(5,5))
    disp = disp.plot(include_values=True, cmap='viridis', ax=ax, xticks_rotation='horizontal')    
    plt.grid(False)
    plt.show()
    return

def get_performance_df(label_actual, label_pred, index_label):
    """
    Function to calculate performance metrics for model.
    Includes precision, recal, F1, & support.
    """
    performance_report = classification_report(label_actual, label_pred, output_dict=True)
    performance_report = performance_report['macro avg']
    result_table = pd.DataFrame(performance_report, index = [index_label])
    result_table = result_table.reset_index().rename(columns = {'index':'model'})
    return result_table

def baseline_models(
    x_train, 
    y_train, 
    x_test, 
    y_test,
    do_smote=True,
    show_confusion_matrix=False,
    show_score_dataframe=False):
    """
    Function that trains and makes predictions using 5 of the classifiers went over during the class.
    Meant as a helper function for easier testing of different modeling pipelines.
    """

    #  do_smote
    if do_smote == True:
        sm = SMOTE(random_state=42)
        x_train, y_train = sm.fit_resample(x_train, y_train)

    # K-Nearest Neighbors
    knn = KNeighborsClassifier()
    knn.fit(x_train, y_train)
    pred_labels_knn  = knn.predict(x_test)
    score_knn = get_performance_df(y_test, pred_labels_knn,'Knn')
    
    # Logistic Regression
    lm = LogisticRegression()
    lm.fit(x_train, y_train)
    pred_labels_lr  = lm.predict(x_test)
    score_lr = get_performance_df(y_test, pred_labels_lr,'Logistic Regression')
        
    # Bernoulii Naive Bayes
    bnb = BernoulliNB()
    bnb.fit(x_train, y_train)
    pred_labels_bnb  = bnb.predict(x_test)
    score_bnb = get_performance_df(y_test, pred_labels_bnb,'Bernoulli Naive Bayes')    
        
    # Gaussian Naive Bayes
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    pred_labels_gnb  = gnb.predict(x_test)
    score_gnb = get_performance_df(y_test, pred_labels_gnb,'Gaussian Naive Bayes')    

    # Random Forest
    rf = RandomForestClassifier(random_state=0)
    rf.fit(x_train, y_train)
    pred_labels_rf  = rf.predict(x_test)
    predictions_posterior_rf = rf.predict_proba(x_test)
    score_rf = get_performance_df(y_test, pred_labels_rf,'Random Forest')   
    
    #Decision Tree
    dt = DecisionTreeClassifier()
    dt.fit(x_train, y_train)
    pred_labels_dt = dt.predict(x_test)
    score_dt = get_performance_df(y_test, pred_labels_dt,'Decision Tree')

    #Gradient Boosting Classifier
    gb = GradientBoostingClassifier()
    gb.fit(x_train, y_train)
    pred_labels_gb = gb.predict(x_test)
    score_gb = get_performance_df(y_test, pred_labels_gb,'Gradient Boosting Classifier')
    
#     #XGBoost
#     xgboost = xgb.XGBClassifier()
#     xgboost.fit(x_train, y_train)
#     pred_labels_xgboost = gbt.predict(x_test)
#     score_xgboost = get_performance_df(y_test, pred_labels_xgboost,'XGBoost')
    
    # make dataframe with scores
    scores = pd.concat([score_knn, score_lr, score_bnb, score_gnb, score_rf, score_dt, score_gb])
    scores = scores.sort_values(by = 'recall', ascending=False)
    
    if show_score_dataframe:
        display(scores.style.set_table_attributes('style="font-size: 17px"').hide_index())
    
    if show_confusion_matrix:
        print('\nK-Nearest Neighbors Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_knn)
        print('Logistic Regression Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_lr)
        print('Bernoulli Naive Bayes Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_bnb)
        print('Gaussian Naive Bayes Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_gnb)
        print('Random Forest Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_rf)
        print('Decision Tree Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_dt)
        print('Gradient Boosting Confusion Matrix')
        plot_confusion_matrix(y_test, pred_labels_gbt)
#         print('XGBoost Confusion Matrix')
#         plot_confusion_matrix(y_test, pred_labels_xgboost)

    return scores

### Baseline models (just depression screener)

In [12]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = all_columns[0:10] # just depression screener
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2729, 10)
(683, 10)
(2729,)
(683,)


In [13]:
baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model[['model','precision','recall','f1-score']]

,model,precision,recall,f1-score
0,Bernoulli Naive Bayes,0.609595,0.672248,0.627428
0,Gaussian Naive Bayes,0.619179,0.657007,0.633433
0,Decision Tree,0.625325,0.581750,0.596221
0,Logistic Regression,0.653195,0.524981,0.525183
0,Random Forest,0.552096,0.512648,0.506990
0,Knn,0.516733,0.506154,0.500418
0,Gradient Boosting Classifier,0.522612,0.502593,0.485650


In [14]:
baseline_model_with_smote = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model_with_smote[['model','precision','recall','f1-score']]

,model,precision,recall,f1-score
0,Logistic Regression,0.590391,0.687960,0.598980
0,Bernoulli Naive Bayes,0.587726,0.677093,0.597050
0,Gaussian Naive Bayes,0.600932,0.675809,0.617413
0,Knn,0.587408,0.638338,0.600637
0,Gradient Boosting Classifier,0.625325,0.581750,0.596221
0,Decision Tree,0.553557,0.545564,0.548803
0,Random Forest,0.510287,0.505500,0.502845


### Baseline models (all variables)

In [30]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom,
    columns = all_columns
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2729, 33)
(683, 33)
(2729,)
(683,)


In [16]:
baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
baseline_model[['model','precision','recall','f1-score']]

,model,precision,recall,f1-score
0,Bernoulli Naive Bayes,0.621495,0.689608,0.641671
0,Gaussian Naive Bayes,0.615217,0.665439,0.631984
0,Decision Tree,0.576176,0.567952,0.571595
0,Gradient Boosting Classifier,0.619722,0.523357,0.523049
0,Logistic Regression,0.588745,0.515895,0.510511
0,Knn,0.567623,0.514271,0.508729
0,Random Forest,0.652065,0.512490,0.501503


In [17]:
baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
baseline_model[['model','precision','recall','f1-score']]

,model,precision,recall,f1-score
0,Logistic Regression,0.590836,0.696392,0.597041
0,Gaussian Naive Bayes,0.586315,0.687306,0.590851
0,Bernoulli Naive Bayes,0.597820,0.673374,0.613396
0,Knn,0.561444,0.643814,0.550770
0,Gradient Boosting Classifier,0.719203,0.580624,0.607471
0,Decision Tree,0.557711,0.567455,0.561674
0,Random Forest,0.649332,0.543310,0.554640


### Test Different Train/Test Proportion

Just using depression screener cols

In [18]:
# compare performance with various train splits from [20%, 80%]
test_size_props = np.arange(0.2,0.8,0.01)

In [19]:
### Without SMOTE

# create base table
model_perf = pd.DataFrame(columns = ['model','precision','recall','f1-score','support'])

# iterate across prop splits and save performance
for prop in test_size_props:

    x_train, x_test, y_train, y_test = get_model_data(
        original_df = cdc_survey_pmom,
        columns = all_columns[0:10],
        test_size_prop = prop
    )
    
    mod_data = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
    mod_data['test_size_prop'] = prop
    model_perf = model_perf.append(mod_data)
    
### With SMOTE

# create base table
model_perf_smote = pd.DataFrame(columns = ['model','precision','recall','f1-score','support'])

# iterate across prop splits and save performance
for prop in test_size_props:

    x_train, x_test, y_train, y_test = get_model_data(
        original_df = cdc_survey_pmom,
        columns = all_columns[0:10],
        test_size_prop = prop
    )
    
    mod_data = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
    mod_data['test_size_prop'] = prop
    model_perf_smote = model_perf_smote.append(mod_data)    

In [20]:
alt.Chart(model_perf).mark_line().encode(
    x='test_size_prop:Q',
    y='f1-score:Q',
    color='model:N',
)

alt.Chart(...)

In [21]:
alt.Chart(model_perf_smote).mark_line().encode(
    x='test_size_prop:Q',
    y='f1-score:Q',
    color='model:N',
)

alt.Chart(...)

### Test Different # of columns

#### Get column feature importances when including all

In [31]:
def get_random_forest_n_top_features(train_data=x_train, 
                                     train_labels=y_train):
    """
    A model that trains a random forest model and returns the feature importances as a dataframe.
    The index of the row corresponds to the column index in the initial array
    """
    
    # fit random forest and print accuracy
    rf = RandomForestClassifier(random_state=0)
    rf.fit(train_data, train_labels)
   
    # list feature importances
    # Get numerical feature importances
    importances = list(rf.feature_importances_)
    feature_importances = pd.DataFrame({'Feature': all_columns,'Coefficient': importances})
    feature_importances = feature_importances.sort_values('Coefficient',ascending=False)
    
    return feature_importances

In [33]:
feature_importances = get_random_forest_n_top_features()
feature_importances

,Feature,Coefficient
28,age_in_years,0.092835
19,count_minutes_moderate_sedentary_activity,0.069866
11,alcoholic_drinks_past_12mo,0.056657
1,feeling_down_depressed_hopeless,0.048181
12,how_healthy_is_your_diet,0.040809
26,monthly_poverty_index,0.040175
9,difficult_doing_daytoday_tasks,0.039431
27,count_hours_worked_last_week,0.038932
29,education_level,0.038772
13,count_lost_10plus_pounds,0.038085


#### Test model performance when using top N features

In [34]:
### Without SMOTE

model_perf_nfeatures = pd.DataFrame(columns = ['model','precision','recall','f1-score','support'])

for number_of_features in list(range(1,len(all_columns))):
    
    # features to use
    # use top N features
    features = list(feature_importances.Feature[:number_of_features])
    
    x_train, x_test, y_train, y_test = get_model_data(
        original_df = cdc_survey_pmom,
        columns = features,
        test_size_prop = 0.2
    )
    
    mod_data = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
    mod_data['n_features'] = number_of_features
    model_perf_nfeatures = model_perf_nfeatures.append(mod_data)    
    
### With SMOTE    
    
model_perf_nfeatures_smote = pd.DataFrame(columns = ['model','precision','recall','f1-score','support'])

for number_of_features in list(range(1,len(all_columns))):
    
    # features to use
    # use top N features
    features = list(feature_importances.Feature[:number_of_features])
    
    x_train, x_test, y_train, y_test = get_model_data(
        original_df = cdc_survey_pmom,
        columns = features,
        test_size_prop = 0.2
    )
    
    mod_data = baseline_models(x_train, y_train, x_test, y_test, do_smote=True)
    mod_data['n_features'] = number_of_features
    model_perf_nfeatures_smote = model_perf_nfeatures_smote.append(mod_data)        

In [35]:
alt.Chart(model_perf_nfeatures).mark_line().encode(
    x='n_features:Q',
    y='f1-score:Q',
    color='model:N',
)

alt.Chart(...)

In [36]:
alt.Chart(model_perf_nfeatures_smote).mark_line().encode(
    x='n_features:Q',
    y='f1-score:Q',
    color='model:N',
)

alt.Chart(...)

#### Test model performance when using N features randomly selected

In [37]:
model_perf_nfeatures_rand = pd.DataFrame(columns = ['model','precision','recall','f1-score','support'])

# randomly sample N features
for number_of_features in list(range(1,len(all_columns)+1)):

    # repeat each N 10 times
    for repeat in range(10):
        columns_to_use = random.sample(all_columns, number_of_features)

        x_train, x_test, y_train, y_test = get_model_data(
            original_df = cdc_survey_pmom,
            columns = columns_to_use,
            test_size_prop = 0.2
        )

        mod_data = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
        mod_data['n_features'] = number_of_features
        model_perf_nfeatures_rand = model_perf_nfeatures_rand.append(mod_data)      

In [38]:
model_perf_nfeatures_rand

,model,precision,recall,f1-score,support,n_features
0,Gaussian Naive Bayes,0.588157,0.541844,0.550559,683,1.0
0,Knn,0.594407,0.537628,0.545475,683,1.0
0,Logistic Regression,0.450952,0.500000,0.474211,683,1.0
0,Bernoulli Naive Bayes,0.450952,0.500000,0.474211,683,1.0
0,Random Forest,0.450952,0.500000,0.474211,683,1.0
...,...,...,...,...,...,...
0,Decision Tree,0.549324,0.547344,0.548283,683,33.0
0,Gradient Boosting Classifier,0.619722,0.523357,0.523049,683,33.0
0,Logistic Regression,0.588745,0.515895,0.510511,683,33.0
0,Knn,0.567623,0.514271,0.508729,683,33.0


In [39]:
# model_perf_nfeatures_rand[(model_perf_nfeatures_rand['model'] == "Knn") & (model_perf_nfeatures_rand['n_features'] == 1)]

In [40]:
alt.Chart(model_perf_nfeatures_rand).mark_point().encode(
    x='n_features:Q',
    y='f1-score:Q',
    color='model:N',
).facet(
    facet='model:N',
    columns=2
)

alt.FacetChart(...)

### Test Using Additional Survey Years

In [50]:
cdc_survey_v2 = pd.read_csv('../data/cdc_nhanes_survey_responses_clean_all_years.csv')

# filter to pregnant moms
cdc_survey_pmom_v2 = cdc_survey_v2[cdc_survey_v2['has_been_pregnant'] == 1]
# cdc_survey_pmom_v2

In [51]:
cdc_survey_pmom_v2.groupby(['folder'])['MDD'].sum()

folder
2013_2014         204
2015_2016         200
2017_march2020    348
Name: MDD, dtype: int64

In [57]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom_v2,
    columns = all_columns[0:10] # just depression screener
)

baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
print('Baseline Models Using 10 Dep Screener Cols (2013 - 2020)')
baseline_model[['model','precision','recall','f1-score']]

Baseline Models Using 10 Dep Screener Cols (2013 - 2020)


,model,precision,recall,f1-score
0,Bernoulli Naive Bayes,0.614506,0.675232,0.632390
0,Gaussian Naive Bayes,0.618641,0.660506,0.633699
0,Decision Tree,0.596242,0.562511,0.572964
0,Random Forest,0.688099,0.543840,0.555191
0,Logistic Regression,0.685605,0.538270,0.546401
0,Knn,0.655004,0.534046,0.539683
0,Gradient Boosting Classifier,0.719752,0.519854,0.513698


In [55]:
x_train, x_test, y_train, y_test = get_model_data(
    original_df = cdc_survey_pmom_v2[cdc_survey_pmom_v2['folder'] == "2017_march2020"],
    columns = all_columns[0:10] # just depression screener
)

baseline_model = baseline_models(x_train, y_train, x_test, y_test, do_smote=False)
print('Baseline Models Using 10 Dep Screener Cols (2017-2020)')
baseline_model[['model','precision','recall','f1-score']]

Baseline Models Using 10 Dep Screener Cols (2017-2020)


,model,precision,recall,f1-score
0,Bernoulli Naive Bayes,0.609595,0.672248,0.627428
0,Gaussian Naive Bayes,0.619179,0.657007,0.633433
0,Decision Tree,0.567399,0.542002,0.548767
0,Logistic Regression,0.653195,0.524981,0.525183
0,Random Forest,0.552096,0.512648,0.506990
0,Knn,0.516733,0.506154,0.500418
0,Gradient Boosting Classifier,0.522612,0.502593,0.485650
